--- 

# **코드 설명**

---

- 파 일 명 : 블랙 프라이데이 판매 예측 <br>
- 시작날짜 : 2021.11.25 <br>
- 수정날짜 : 2021.11.25 <br>
- 작 성 자 : 김혁진 <br>
- 작성주제 : DataHackaton / 블랙 프라이데이 판매 예측 <br>

--- 

- **순서** <br>
  0. 기본설정 <br>
    0.0. Google Drive Mount <br>
    0.1. GPU 사용 <br>
    0.2. Import Modules <br>
    0.3. Initial Values
    0.4. Set Off the Warning <br>
    0.5. User Defined Function <br>

  1. Data Load <br>

  2. EDA
  
---

- **참조**

  (1) 대회 홈페이지 : [DataHackaton](https://datahack.analyticsvidhya.com/contest/black-friday/#MySubmissions) <br>
  (2) 하이퍼 파리미터 설명 : [Naver Blog](https://blog.naver.com/wideeyed/221333529176) <br>
  (3) Class문 설명 : [Github](https://zzsza.github.io/development/2020/07/05/python-class/) <br>
  (4) GPU 설정 : [Medium](https://medium.com/@am.sharma/lgbm-on-colab-with-gpu-c1c09e83f2af) <br>
  (5) RAM 모두사용으로 세션다운 : [Tistory](https://somjang.tistory.com/entry/Google-Colab-%EC%9E%90%EC%A3%BC%EB%81%8A%EA%B8%B0%EB%8A%94-%EB%9F%B0%ED%83%80%EC%9E%84-%EB%B0%A9%EC%A7%80%ED%95%98%EA%B8%B0)

---

- **고려사항** <br>
  (1) AutoEncoder로 파생변수 생성해보기

---

># **0. 기본설정**

### markdown : tabular left align

In [1]:
%%html
<style>
    table {float:left}
</style>

In [2]:
# # 0.0.1 Google Drive Mount
# # (Google Drive 사용 시 설정)
# from google.colab import drive
# drive.mount('/content/drive', force_remount = True) # 새로운 창에서 key 를 받아서 입력해야합니다. 

# # 0.0.2. 쥬피터 노트북 화면 넓게 사용
#출처: https://taehooh.tistory.com/entry/Jupyter-Notebook-주피터노트북-화면-넓게-쓰는방법 [블로그]
from IPython.core.display import display, HTML 
display(HTML("<style>.container { width:80% !important; }</style>"))

# # 0.0.3. 메모리 에러
# https://growingsaja.tistory.com/477

In [ ]:
# # 0.1. GPU 사용 (6분)
# !git clone --recursive https://github.com/Microsoft/LightGBM
# !mkdir build
# %cd /content/LightGBM
# !cmake -DUSE_GPU=1 #avoid ..
# !make -j$(nproc)
# !sudo apt-get -y install python-pip
# !sudo -H pip install setuptools pandas numpy scipy scikit-learn -U
# %cd /content/LightGBM/python-package

In [ ]:
# !pip uninstall pandas -y
# !pip uninstall numpy  -y
# !pip uninstall lightgbm -y

# !pip install pandas==1.1.0
# !pip install numpy==1.21.2
# !pip install -U scikit-learn
# !pip install lightgbm --install-option=--gpu

# !pip install pandasql
# !pip install seaborn
# !pip install plotnine
# !pip install pandasql

In [3]:
# 0.2. Import Modules

# jupyter notebook 전용
from tqdm.notebook import tqdm

import pandas as pd
import numpy as np
# from tqdm import tqdm
import warnings
import random
import os
import time

from collections import Counter as cnt

# plotting
import seaborn as sns
sns.set(rc={'figure.figsize':(11.7, 8.27)})
sns.set_style('whitegrid')

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [11.7, 8.27] # [15, 10] # [11.7,8.27] - A4 size

from plotnine import *
from pandasql import sqldf
sql = lambda q: sqldf(q, globals())

# modeling
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
# import lightgbm
# !pip install lightgbm --install-option=--gpu --install-option="--opencl-include-dir=/usr/local/cuda/include/" --install-option="--opencl-library=/usr/local/cuda/lib64/libOpenCL.so"

In [9]:
# 0.3. Initial Values
SEED = 777

# jupyter.notebook : 'os.getcwd() + '/DAT/블랙 프라이데이 판매 예측/''
# google.colab : '/content/drive/MyDrive/Python/4. 블랙프라이데이 판매예측/DAT/'
DATA_PATH = os.getcwd() + '/DAT/1. 블랙 프라이데이 판매 예측(데이터 해커톤)/'
SCALE = False

In [5]:
# 0.4. Set Off the Warning
pd.set_option('mode.chained_assignment', None)
warnings.filterwarnings(action='ignore')

In [6]:
# 0.5. User Defined Function
# 0.5.1. Seed Fix
def seed_everything(seed: int = 1):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    # torch.manual_seed(seed)
    # torch.cuda.manual_seed(seed)  # type: ignore
    # torch.backends.cudnn.deterministic = True  # type: ignore
    # torch.backends.cudnn.benchmark = True  # type: ignore
    
seed_everything(SEED)

# 0.5.2. View all columns
def View(data):

    pd.set_option('display.max_rows', 500)
    pd.set_option('display.max_columns', 500)
    pd.set_option('display.width', 1000)
    
    print(data)

    pd.set_option('display.max_rows', 0)
    pd.set_option('display.max_columns', 0)
    pd.set_option('display.width', 0)

# 0.5.3. minmax function
def minmax(x):
    return min(x),max(x)

# 0.5.4. 컬럼dict에서 target 제거
def removekey(dict, key):
    tmp = dict.copy()
    del tmp[key]
    return tmp

# 0.5.5. 각 컬럼의 missing 개수를 파악하는 함수
def missing_column_check(data):
    num_na = []
    for col_nm in data.columns:
        data[col_nm] = data[col_nm].astype(COL_TYPE[col_nm])

    # str인 경우에는 blank(공백)도 있는지 확인
    if COL_TYPE[col_nm]=='str':
        num_na_x = data[col_nm].str.strip().isnull().sum() + sum(data[col_nm].str.strip()=='')
    
    # numeric인 경우에는 null의 개수만 확인
    else:
        num_na_x = data[col_nm].isnull().sum()
    
    num_na = num_na + [num_na_x]

    return(num_na)

In [7]:
# 최근버전 확인
import sys
print(sys.version)

3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]


># **1. Data Load**

|변수명 | 변수정보 |
|:---:|:---|
| User_ID | 사용자 ID |
| Product_ID | 제품 ID |
| Gender | 사용자의 성별 |
| Age | 나이(구간) |
| Occupation | 직업(마스킹됨) |
| City_Category | 도시의 범주(A,B,C) |
| Stay_In_Current_City_Years | 현재 도시에 체류한 기간 |
| Marital_Status | 결혼 여부 |
| Product_Category_1 | 제품 카테고리 (마스킹됨) |
| Product_Category_2 | 제품 카테고리2 (마스킹됨,다른 카테고리에도 속할 수 있음) |
| Product_Category_3 | 제품 카테고리3 (마스킹됨,다른 카테고리에도 속할 수 있음) |
| Purchase | 구매금액(Target) |

In [11]:
# column type 변환
DTYPE_DICT = {
    'User_ID'                     : str,
    'Product_ID'                  : str,
    'Gender'                      : str,
    'Age'                         : str,
    'Occupation'                  : str,
    'City_Category'               : str,
    'Stay_In_Current_City_Years'  : str,
    'Marital_Status'              : str,
    'Product_Category_1'          : str,
    'Product_Category_2'          : str,
    'Product_Category_3'          : str,
    'Purchase'                    : float
}

# Train Data Load (550,068 rows, 12 columns)
train = pd.read_csv(DATA_PATH + "train.csv", dtype = DTYPE_DICT)
train

# DATA_PATH + "train.csv"

'C:\\Users\\My/DAT/1. 블랙 프라이데이 판매 예측(데이터 해커톤)/train.csv'

In [ ]:
# Product_Category_2, 3은 null값이 있음
train.info()

In [ ]:
# Test Data Load (233,599 rows, 11 columns - without target)
test = pd.read_csv(DATA_PATH + 'test.csv', dtype = removekey(DTYPE_DICT,'Purchase'))
test

<br> </br>

># **2. EDA (1-Dimension)**

## 2.1. Overall View

### 2.1.1. vs oneself : Target인 Purchase를 제외하고는 모두 Categorical 변수 → Count 확인

In [ ]:
# 기존 max_rows
org_max_r = 60

# 각 변수 level 별 count/mean_count 확인
pd.set_option('max_rows',4)
for iter,col in enumerate(train.columns[:-1]):
    cnt = train[col].value_counts(dropna=False).reset_index()  # NA포함 count
    d   = pd.DataFrame({col : cnt['index'],'CNT' : cnt[col]})  # to DataFrame
  
    d['mean'] = str(round(d['CNT'].agg(np.mean),1))            # mean of counts
    d['mean'][1:] = ''

    print('-'*60, '\n', d)
    del d

pd.set_option('max_rows',org_max_r)

|변수명|설명|
|:---:|:---|
|User_ID | 최대 1,026번이나 구매한 사람이 있고, 평균적으로 93번정도 구매했음|
|Product_ID | 최대 1,880번이나 구매된 제품이 있고, 평균적으로 151번정도 구매됨|
|Gender | 남자가 75.3%, 여자가 24.7% -> 많이/적게 산 제품의 남녀 비중 확인|
|Age | 26-35세, 36-45세 가 많았음. -> 위와 동일|
|Occupation | 마스킹이 제대로 되어, 순서에 의미가 없음 -> 많이/적게 산 직업의 구매비중이 높/낮은지, 구매금액이 높/낮은지 확인|
|City_Category | 은 B,C,A순으로 구매가 많음 -> 위와 동일|
|Stay_In_Current_City_Years | 적게 살았을수록 구매회수가 많고, 적게 살았을수록 구매회수가 적음 -> 초기에 살 때 많이 사야하는 품목들이 많은 것으로 보임|
|Marital_Satatus | 미혼인 경우 구매회수가 많음 (59.0%)|
|Product_Category | 제품 카테고리도 마스킹이 제대로 되어, 순서에 의미가 없음|

<br> </br>
### 2.1.2. vs Target : 전체적인 산점도 확인 (Categorical 이더라도 Purchase간의 관계확인)

In [ ]:
# 전체적으로 보기 위해, pairplot을 사용
pairplot_df = train.copy()

# 필요없는 변수 제거
for del_col in ['User_ID','Product_ID','Product_Category_2','Product_Category_3']:
    del pairplot_df[del_col]

# str -> int 변환
for col_str2int in pairplot_df.columns:
    pairplot_df[col_str2int] = pairplot_df[col_str2int].astype('category').cat.codes

# pairplot
sns.pairplot(pairplot_df,
             x_vars=['Gender','Age','Occupation','City_Category','Stay_In_Current_City_Years','Marital_Status','Product_Category_1'],
             y_vars=['Purchase'])

- 전체적으로 Purchase 값의 최소,최대값이 각 변수마다 차이가 나지않는 것으로 보임 (pairplot으로는 상세하게 확인하기 힘드니, 아래에서 상세하게 확인)
- Product_Category_1은 값이 각 값마다 구간별로 흩어져 있는 것 같은데.. 확인필요함

<br> </br>
## 2.2. Categorical Variable

### 2.2.1. User_ID

In [ ]:
# 1. 사용자   - 5,891명 (최소,최대) : (6,1026)번 구매함
#               한명의 사람이 많이 구매하는 경우가 있음
#               구매 회수는 포아송분포랑 비슷한 것 같음

print('전체 구매자 수 :',train['User_ID'].value_counts().count(), '명\n')

# (1) Q0 ~ Q4
print('Q0-Q4 : ',
      train['User_ID'].value_counts().quantile([0,0.25,0.50,0.75,1]).values)

# (2) Q3보다 많이 구매한 사람은 1467명
q1_q3 = train['User_ID'].value_counts().quantile([0.25,0.75]).values

print('Q1보다 적게 구매한 사람 :',
      train['User_ID'].value_counts()[train['User_ID'].value_counts()<q1_q3[0]].shape[0], '명')
print('Q3보다 많이 구매한 사람 :',
      train['User_ID'].value_counts()[train['User_ID'].value_counts()>q1_q3[1]].shape[0], '명')
print('(수리통계학적으로 Q1~Q3는 50%)\n')

# (3) upper whisker보다 많이 구매한 사람은 473명
from matplotlib.cbook import boxplot_stats
stats = boxplot_stats(train['User_ID'].value_counts())[0]
print('Lower/Upper Whisker :',stats['whislo'], stats['whishi'])

print('Upper Whisker보다 많이 구매한 사람 :',
      train['User_ID'].value_counts()[train['User_ID'].value_counts()>stats['whishi']].shape[0],
      '(',round(train['User_ID'].value_counts()[train['User_ID'].value_counts()>stats['whishi']].shape[0]/train['User_ID'].value_counts().count(),3)*100,'%)')

# boxplot
def box_hist_plot(val):
    f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, 
                                    gridspec_kw={"height_ratios": (.15, .85)})

    sns.boxplot(val, ax=ax_box)
    sns.distplot(val, ax=ax_hist)

    ax_box.set(yticks=[])
    sns.despine(ax=ax_hist)
    sns.despine(ax=ax_box, left=True)
    
box_hist_plot(train['User_ID'].value_counts())
plt.axvline(x=stats['whishi'], color='r', linestyle=':')
plt.show()

### 2.2.2. Product_ID

In [ ]:
tmp = train.copy()

# 첫째자리는 P로 동일함
print('첫째 자리 :',cnt([x[0] for x in tmp['Product_ID']]), '\n')

# 8,9자리가 있음
tmp['Product_ID_len'] = [len(x) for x in tmp['Product_ID']]
print('전체 글자 수 :\n',tmp['Product_ID_len'].value_counts(), '\n')

# P00123456로 유추됨!

In [ ]:
# 8자리의 substr(2,4)은 모두 00,
# 9자리의 substr(2,4)도 모두 00

# 앞에서 부터
for i in range(1,9):
    
    cnt_x = pd.DataFrame([x [i:i+2] for x in tmp['Product_ID'][tmp['Product_ID_len']==8]]).value_counts().count()
    cnt_y = pd.DataFrame([x [i:i+2] for x in tmp['Product_ID'][tmp['Product_ID_len']==9]]).value_counts().count()
    
    if cnt_x<10 and cnt_y<10:
    
        print('='*80,'i=',i,'\n',
              pd.DataFrame([x [i:i+2] for x in tmp['Product_ID'][tmp['Product_ID_len']==8]]).value_counts(),'\n','-'*80,'\n',
              pd.DataFrame([x [i:i+2] for x in tmp['Product_ID'][tmp['Product_ID_len']==9]]).value_counts())

In [ ]:
# 뒤에서 부터
for i in range(0,10):
    
#     if i==1:
#         print('i=',i,'P00123456'[-2:])
#     else:
#         print('i=',i,'P00123456'[-i:-i+2])

    if i==1:
        
        cnt_x = pd.DataFrame([x [-2:] for x in tmp['Product_ID'][tmp['Product_ID_len']==8]]).value_counts().count()
        cnt_y = pd.DataFrame([x [-2:] for x in tmp['Product_ID'][tmp['Product_ID_len']==9]]).value_counts().count()
        
        if cnt_x<10 and cnt_y<10:
        
            print('='*80,'i=',i,'\n',
                  pd.DataFrame([x [-2:] for x in tmp['Product_ID'][tmp['Product_ID_len']==8]]).value_counts(),'\n','-'*80,'\n',
                  pd.DataFrame([x [-2:] for x in tmp['Product_ID'][tmp['Product_ID_len']==9]]).value_counts())
    else:
        
        cnt_x = pd.DataFrame([x [-i:-i+2] for x in tmp['Product_ID'][tmp['Product_ID_len']==8]]).value_counts().count()
        cnt_y = pd.DataFrame([x [-i:-i+2] for x in tmp['Product_ID'][tmp['Product_ID_len']==9]]).value_counts().count()
        
        if cnt_x<10 and cnt_y<10:

            print('='*80,'i=',i,'\n',
            pd.DataFrame([x [-i:-i+2] for x in tmp['Product_ID'][tmp['Product_ID_len']==8]]).value_counts(),'\n','-'*80,'\n',
            pd.DataFrame([x [-i:-i+2] for x in tmp['Product_ID'][tmp['Product_ID_len']==9]]).value_counts())

- 앞/뒤로 2자리 씩 봤을 때, P 이후에 0을 넣어주는게 제일 알맞아보임
- 이렇게하면, Product_ID의 형태는 'P00123456'의 형태로 보임 (숫자가 커지면 'P12345678'으로 바뀔 것으로 예상)

In [ ]:
# 맨앞자리에 0을 넣어줌
tmp['Product_ID'][tmp['Product_ID_len']==8] = [x[0] + '0' + x[1:] for x in tmp['Product_ID'][tmp['Product_ID_len']==8]]

<br> </br>
Product_ID를 수정한 이후, i번째 자리에서의 k자리 조합을 확인 (특별하게 부여된 패턴이 있는지 확인) \
위와는 다르게, 굳이 모든 패턴을 다 볼 필요는 없고, Freq만 확인 \
(Product_Category_1이 20가지인데, 이 이상이 되는 건 확인하지 않기로 함)

In [ ]:
# P00123456의 형태
n_total = 0
for k in range(2,7):      # 2~7자리 조합
    print('-'*50,'k=',k)
    for i in range(2,9-k+1):  # 1~6번째 자리
        
        freq = pd.DataFrame([x [i:i+k] for x in tmp['Product_ID']]).value_counts().shape[0]
        
        n_total += +1
        blank = ' ' if n_total < 10 else ''
        
        print(f'n_total={n_total} {blank}/ i,k,freq = {i},{k},{freq}')
        
        if n_total == 6+5+4+3+2:
            print('-'*50)

나눠볼만한 Cases : [i,k,cases] = 
- 2자리 : [2,2,4 case],[6,2,13 case],[7,2,6 case] 
- 3자리 : [6,3,15 case] 

In [ ]:
tmp['Product_ID_34'] = [x[2:4] for x in tmp['Product_ID']]
tmp['Product_ID_78'] = [x[6:8] for x in tmp['Product_ID']]
tmp['Product_ID_89'] = [x[7:9] for x in tmp['Product_ID']] # 딱 나뉨 : 대부분 42라서 그럼
tmp['Product_ID_79'] = [x[6:9] for x in tmp['Product_ID']]

pd.crosstab(tmp['Product_ID_89'],tmp['Product_Category_1'].fillna('-999'))

#

In [ ]:
plt.scatter(train['Product_Category_1'].astype('category').cat.codes,
            train['Purchase'],
            s = 3)


plt.axhline(5500, color='red', linestyle=':', linewidth=2)

In [ ]:
sub = train[[x=='1' for x in train['Product_Category_1']]]

# 1번 카테고리 그룹
sub_1 = sub[[            x<5000  for x in sub['Product_Category_1'].astype(int)]]
sub_2 = sub[[x>5000  and x<10000 for x in sub['Product_Category_1'].astype(int)]]
sub_3 = sub[[x>10000 and x<15000 for x in sub['Product_Category_1'].astype(int)]]
sub_4 = sub[[x>15000 and x<17500 for x in sub['Product_Category_1'].astype(int)]]
sub_5 = sub[[x>17500             for x in sub['Product_Category_1'].astype(int)]]

# 2번 카테고리 그룹, ...

for iter in range(1,6):
    if len(eval(f'sub_{iter}')['Product_Category_2'].value_counts()) > 0:
        print(display(pd.crosstab(eval(f'sub_{iter}')['Product_Category_1'], 
                                  eval(f'sub_{iter}')['Product_Category_2'].fillna('missing'),
                                  margins = True)))

In [ ]:
# Purchase 변수에 대해서 histgram + density plot
def density_plot(data, var, binwidth, hue = None):

    from matplotlib.ticker import PercentFormatter

    fig, ax1 = plt.subplots()
    sns.histplot(data = data, x = var, hue = hue,
                 kde = True, stat = 'probability', 
                 color = 'lightskyblue', label = 'Probabilities',
                 binwidth = binwidth, ax = ax1,)
    ax2 = ax1.twinx()
    sns.kdeplot(data = data, x = var, hue = hue,
                color = 'red', label = 'kde density',
                lw = 2, ax = ax2)
    ax2.set_ylim(0, ax1.get_ylim()[1] / binwidth)                  # similir limits on the y-axis to align the plots
    ax2.yaxis.set_major_formatter(PercentFormatter(1 / binwidth))  # show axis such that 1/binwidth corresponds to 100%

# 어려 분포가 섞여있는걸로 보임
# > 대락 8000, 15500, 19000 쯤에 평균을 가지는 mixed normal dist.
# > 1) 평균 : 8000 , 범위 : 8000
# > 2) 평균 : 15500, 범위 :  
density_plot(train, var='Purchase', binwidth = 1000)

In [ ]:
# mix된 분포를 나누기 위해, 그룹핑해서 볼 변수들
group_var = ['Gender','Age','Occupation','City_Category','Stay_In_Current_City_Years','Marital_Status']

for iter,var in enumerate(tqdm(group_var, position=0, leave=True)):
    g = sns.FacetGrid(data=train, hue=var, height=5, aspect = 2)
    g = g.map(sns.kdeplot, 'Purchase', shade=False)
    g.add_legend()
    g.fig.suptitle(str(iter+1) + '. ' + var, fontsize = 25, y = 1.05)
    
    print(g, '\n')
    del g

# => 나뉘지 않음

In [ ]:
# Product_Category_1로 나눠보기
# _2, _3은 missing이 많아서..

# # fig, ax1 = plt.subplots(1)

# # 전체
# p = sns.FacetGrid(data=train, hue=None, height=5, aspect = 2) #, col_wrap = 2)
# p.map(sns.kdeplot, 'Purchase', shade=False, alpha = 0.5, linewidth = 5, ax = ax1).set(yscale = 'log')

iter_list = [str(x) for x in list(range(0,21))[1:]]
for iter in range(0,5):

    iter_list_x = iter_list[4*iter:4*iter+4]
    sub = train[[x in iter_list_x for x in train['Product_Category_1'].values]]

    g = sns.FacetGrid(data=sub, hue='Product_Category_1', height=5, aspect = 2) #, col_wrap = 2)
    g.map(sns.kdeplot, 'Purchase', shade=False, alpha = 0.5, ax = ax1).set(yscale = 'log')
    g.add_legend()
    g.fig.suptitle('Product_Category_1', fontsize = 25, y = 1.05)

    
# 14,15,16 and 6,7 (x범위 고려o)
# ...

In [ ]:
# 제품구분 별 boxplot
sub = train.copy()
sub['Product_Category_1'] = sub['Product_Category_1'].astype(int)

p1 = sns.boxplot(x='Product_Category_1',y='Purchase',data=sub)

vls = [8000, 15500, 19000]
for vl in vls:
    p1 = plt.axhline(y=vl, color='r', linestyle='--', alpha=0.7)

In [ ]:
import math
# train['Product_Category_1'].value_counts()

sub = train.copy()
sub['Product_Category_1'] = sub['Product_Category_1'].astype(int)
sub['Product_Category_2'] = sub['Product_Category_2'].fillna('-999').astype(int)
sub['Product_Category_3'] = sub['Product_Category_3'].fillna('-999').astype(int)

not_nan_val_2 = [x!=-999 for x in train['Product_Category_2'].fillna('-999').astype(int)]
pd.crosstab(sub['Product_Category_1'][not_nan_val_2], sub['Product_Category_2'][not_nan_val_2].fillna(-999))

not_nan_val_3 = [x!=-999 for x in train['Product_Category_3'].fillna('-999').astype(int)]
pd.crosstab(sub['Product_Category_1'][not_nan_val_3], sub['Product_Category_3'][not_nan_val_3].fillna(-999))

In [ ]:
print(train['Product_Category_1'].value_counts())

In [ ]:
from sklearn.cluster import KMeans 

# k-means clustering 실행
kmeans = KMeans(n_clusters=3)
kmeans.fit(pd.DataFrame(train['Product_Category_1'],train['Purchase']))

kmeans.labels_

train['Product_Category_1_subgroup'] = kmeans.labels_

pd.crosstab(train['Product_Category_1'].astype(int),train['Product_Category_1_subgroup'].astype(int))

In [ ]:
sub = train.copy()
sub['Product_Category_1'] = sub['Product_Category_1'].astype(int)

ggplot(aes(x='Product_Category_1',y='Purchase',color='Gender'), data = sub) +\
  geom_point()

# plt.scatter(sub['Product_Category_1'],sub['Purchase'])

<br></br>

* Continuous Variable : Purchase

#

In [ ]:
# 위에서 정확하게 보지못한 변수들을 bar plot으로 확인
plot_col = ['Age', 'Occupation', 'City_Category', 'Stay_In_Current_City_Years', 'Product_Category_1', 'Product_Category_2', 'Product_Category_3']
plot_df  = train[plot_col]

# 각 변수별 level별 barplot
for iter,col in enumerate(plot_col):

    # na값을 'NA'로 변환
    null_idx = plot_df[col].isnull()
    plot_df[col][null_idx] = 'NA'

    # count 계산
    cnt = plot_df[col].value_counts(dropna=False).reset_index()
    d   = pd.DataFrame({col : cnt['index'],
                      'CNT' : cnt[col]})

    # barplot
    plt.figure()
    plt.bar(d[col],d['CNT'])
    plt.title(str(iter+1) + ' : '+col, fontsize = 20, loc = 'left')
    plt.show()
    print('\n')

In [ ]:
# q1보다크면 1, q2보다 크면 2, q3보다크면 3

### Product_ID vs Else

In [ ]:
# 이전 카테고리2와 동일하면 입력안하는 시스템일 수도 있음
tmp = train.copy()

# user_id + product_id = unique
tmp['product_by_user'] = tmp['User_ID'] + '_' + tmp['Product_ID']
tmp.groupby(['product_by_user'])['product_by_user'].count()

# isnan의 개수에 대한 tabular
tmp['isnan_cate2'] = [x=='-999' for x in tmp['Product_Category_2'].fillna('-999')]
nan_tbl = pd.crosstab(tmp['User_ID'],tmp['isnan_cate2']).reset_index()

# False(nan이 아닌) 개수가 1미만(=0)은 없음
sum(nan_tbl[False]<1), sum(nan_tbl[True]<1)

In [ ]:
# nan이 없는 것들을 살펴보면
nan_tbl[nan_tbl[True]<1]

tmp[tmp['User_ID']==nan_tbl['User_ID'][3]]\
    [['User_ID','Product_ID','Product_Category_2']]

# 뭔가 그런패턴을 알아볼수있는지 잘모르겠음

### 파생변수 생성

In [ ]:
# 파생변수

def new_feature_fn(data):
    data['n_purchase'] = data.groupby('User_ID')['User_ID'].transform('count') # 이건 train, test 합쳐서 하는게 좋아보임
    data['n_purchase_q1'] = train['User_ID'].value_counts().quantile([0.25]).values
    data['n_purchase_q1'] = train['User_ID'].value_counts().quantile([0.25]).values
    
    data['Product_ID_len'] = [len(x) for x in data['Product_ID']]
    
    data['Product_ID'][data['Product_ID_len']==8] = [x[0] + '0' + x[1:] for x in data['Product_ID'][data['Product_ID_len']==8]]
    
    data['Product_ID_34'] = [x[2:4] for x in data['Product_ID']]
    data['Product_ID_78'] = [x[6:8] for x in data['Product_ID']]
    data['Product_ID_89'] = [x[7:9] for x in data['Product_ID']]
    data['Product_ID_79'] = [x[6:9] for x in data['Product_ID']]
    
    del data['Product_ID_len']
    
    return data

new_feature_fn(train)